In [7]:
# import os
# import shutil
# from PIL import Image
# import torch
# import torchvision.transforms as transforms
# import torchvision.models as models

# def predict_image_type(model, image_path):
#     image = Image.open(image_path).convert("RGB")
#     transform = transforms.Compose([
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), 
#     ])
#     image = transform(image).unsqueeze(0)
#     with torch.no_grad():
#         output = model(image)
#         _, predicted_class = torch.max(output, 1)
#     return predicted_class.item()

# def separate_images(source_dir, target_dirs):
#     temp_dir = "/kaggle/working/temp_images"
#     os.makedirs(temp_dir, exist_ok=True)

#     model = models.resnet50(pretrained=True)  

#     class_to_directory = {
#         600: target_dirs[0],
#         463: target_dirs[1],
#         731: target_dirs[2],
#     }

#     for root, _, files in os.walk(source_dir):
#         for file in files:
#             image_path = os.path.join(root, file)
#             predicted_type = predict_image_type(model, image_path)

#             if predicted_type in class_to_directory:
#                 target_path = os.path.join(class_to_directory[predicted_type], file)
#                 shutil.copy(image_path, target_path)
#             else:
#                 print(f"Skipping file '{file}' as predicted_type '{predicted_type}' does not have a target directory.")

#     shutil.rmtree(temp_dir)

# if __name__ == "__main__":
#     source_directory = "/kaggle/input/shoe-boot-sandal/shoe_boot_sandal"
#     target_directories = ["/kaggle/working/shoe-boot-sandal/type1", "/kaggle/working/shoe-boot-sandal/type2", "/kaggle/working/shoe-boot-sandal/type3"]

#     separate_images(source_directory, target_directories)

In [2]:
import os
import shutil
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

src_dir = '/kaggle/input/shoe-boot-sandal/shoe_boot_sandal/'
dst_dir = '/kaggle/working/shoe-boot-sandal/'

image_files = [f for f in os.listdir(src_dir) if f.endswith('.jpg')]

model = ResNet50(weights='imagenet')

for image_file in image_files:
    img_path = os.path.join(src_dir, image_file)
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)
    pred_class = decode_predictions(preds, top=1)[0][0][1]

    if 'boot' in pred_class:
        dst_subdir = 'boot'
    elif 'sandal' in pred_class:
        dst_subdir = 'sandal'
    elif 'shoe' in pred_class:
        dst_subdir = 'shoe'

    os.makedirs(os.path.join(dst_dir, dst_subdir), exist_ok=True)
    shutil.copy(os.path.join(src_dir, image_file), os.path.join(dst_dir, dst_subdir, image_file))


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

1/1 [==============================] - 0s 145ms/step


In [4]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return display(FileLink(f'{download_file_name}.zip'))

In [5]:
download_file('/kaggle/working/shoe-boot-sandal', 'dir1')